In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import rasterio
import rasterio.mask
import numpy as np
import numpy.ma as ma
from rasterio.windows import from_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import os
import shutil
from zipfile import ZipFile


In [21]:
all_geom = gpd.read_file('farm_simple.geojson')
pad_geom = all_geom[all_geom.name == 'Padded box']
pad_geom_m = pad_geom.to_crs('epsg:32616')

In [31]:
with rasterio.open('./data/S2A_MSIL2A_20200418T164901_N0214_R026_T16TCM_20200418T210937.SAFE/GRANULE/L2A_T16TCM_A025191_20200418T165421/IMG_DATA/R10m/T16TCM_20200418T164901_B04_10m.jp2', 'r') as b4:
    b4_img, b4_transform = rasterio.mask.mask(b4, pad_geom_m.geometry, crop=True)
    b4_out_meta = b4.meta.copy()
    b4_out_meta.update({'driver': 'GTiff',
                     'height': b4_img.shape[1],
                     'width': b4_img.shape[2],
                     'transform': b4_transform})
    red = b4_img.astype(float)

with rasterio.open('B4_masked.tiff', 'w', **b4_out_meta) as dest:
    dest.write(b4_img)

with rasterio.open('./data/S2A_MSIL2A_20200418T164901_N0214_R026_T16TCM_20200418T210937.SAFE/GRANULE/L2A_T16TCM_A025191_20200418T165421/IMG_DATA/R10m/T16TCM_20200418T164901_B08_10m.jp2', 'r') as b8:
    b8_img, b8_transform = rasterio.mask.mask(b8, pad_geom_m.geometry, crop=True)
    b8_out_meta = b8.meta.copy()
    b8_out_meta.update({'driver': 'GTiff',
                        'height': b8_img.shape[1],
                        'width': b8_img.shape[2],
                        'transform': b8_transform})
    nir = b8_img.astype(float)

with rasterio.open('B8_masked.tiff', 'w', **b8_out_meta) as dest:
    dest.write(b8_img)


ndvi = (nir - red)/(nir + red)
b4_out_meta.update({'dtype': rasterio.float64})

with rasterio.open('NDVI_masked.tiff', 'w', **b4_out_meta) as dest:
    dest.write(ndvi.astype(rasterio.float64))

/Users/MarkNyberg/PycharmProjects/sentinel_processing/venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [45]:
print(np.nanmean(ndvi))
print(np.nanmax(ndvi))
print(np.nanmin(ndvi))

0.3608108486820853
0.9284578696343402
-0.04655493482309125


In [46]:
print(np.count_nonzero(~np.isnan(ndvi)))

45891
